In [3]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

from sklearn.model_selection import train_test_split
from datetime import datetime

from functions.data_filtering import complete_flight_filter, filter_flights, filter_by_bools
from functions.data_processing import get_takeoff_and_landing_directions, prepare_wind_data
from functions.data_loading import get_filtered_data_range, get_wind_direction, get_data_range

device = (torch.device('cuda')
          if torch.cuda.is_available()
          else torch.device('cpu'))

In [4]:
origin = "bergen"
destination = "oslo"
unfiltered_data, file_name = get_data_range(origin, destination, datetime(year=2023, month=1, day=1), datetime(year=2024, month=1, day=1))
print(file_name)
filtered_flights, file_name = get_filtered_data_range(unfiltered_data, file_name, complete_flight_filter(origin, destination))

bergen-oslo-2023-01-01-2024-01-01.pkl


In [5]:
directions = list(get_takeoff_and_landing_directions(filtered_flights))
dataset = pd.DataFrame(directions)
dataset.rename({0: "start_time",  1: "end_time", 2: "start_direction", 3: "end_direction"}, axis=1, inplace=True)
dataset['start_x'], dataset['start_y']  = np.sin(dataset['start_direction']), np.cos(dataset['start_direction'])
dataset['end_x'], dataset['end_y'] = np.sin(dataset['end_direction']), np.cos(dataset['end_direction'])
dataset['start_time'] = pd.to_datetime(dataset['start_time']).dt.round("s")
dataset['end_time'] = pd.to_datetime(dataset['end_time']).dt.round("s")
dataset['start_label'] = ((np.sign(dataset['start_y']) + 1) // 2).astype(int)
dataset['end_label'] = ((np.sign(dataset['end_y']) + 1) // 2).astype(int)

In [33]:
dataset.head(5)

,start_time,end_time,start_direction,end_direction,start_x,start_y,end_x,end_y,start_label,end_label
0,2023-02-07 20:44:04+00:00,2023-02-07 21:30:00+00:00,2.936878,3.423365,0.203288,-0.979119,-0.278058,-0.960564,0,0
1,2023-06-21 04:15:14+00:00,2023-06-21 04:50:10+00:00,2.968032,0.270153,0.172691,-0.984976,0.266879,0.963730,0,1
2,2023-06-25 07:02:26+00:00,2023-06-25 07:36:04+00:00,2.968032,3.425084,0.172691,-0.984976,-0.279710,-0.960085,0,0
3,2023-03-04 08:21:18+00:00,2023-03-04 08:55:38+00:00,6.125815,0.275272,-0.156721,0.987643,0.271809,0.962351,1,1
4,2023-11-29 08:38:32+00:00,2023-11-29 09:18:40+00:00,6.108513,0.276035,-0.173785,0.984784,0.272543,0.962144,1,1


In [7]:
gardermoen = get_wind_direction("GARDERMOEN")
wind_data = prepare_wind_data(gardermoen)

In [32]:
wind_data.head(5)

,wind_direction,wind_speed,x,y,x_scaled,y_scaled
time,,,,,,
2023-01-01 00:00:00+00:00,0.750492,1.200000,0.681998,0.731354,0.818398,0.877624
2023-01-01 00:00:01+00:00,0.749575,1.201167,0.681328,0.731978,0.818388,0.879228
2023-01-01 00:00:02+00:00,0.748659,1.202333,0.680657,0.732602,0.818377,0.880832
2023-01-01 00:00:03+00:00,0.747743,1.203500,0.679985,0.733226,0.818362,0.882437
2023-01-01 00:00:04+00:00,0.746826,1.204667,0.679313,0.733848,0.818346,0.884043


In [9]:
merged_start = pd.merge(wind_data, dataset, how="right", left_on="time", right_on="start_time")
merged_end = pd.merge(wind_data, dataset, how="right", left_on="time", right_on="end_time")
final_dataset = pd.DataFrame({
    'start_wind_x': merged_start['x'],
    'start_wind_y': merged_start['y'],
    'start_wind_speed': merged_start['wind_speed'],
    'end_wind_x': merged_end['x'],
    'end_wind_y': merged_end['y'],
    'end_wind_speed': merged_end['wind_speed'],
    'start_label': dataset['start_label'],
    'end_label': dataset['end_label'],
})

In [31]:
final_dataset.head(5)

,start_wind_x,start_wind_y,start_wind_speed,end_wind_x,end_wind_y,end_wind_speed,start_label,end_label
0,-0.167526,0.985868,1.023733,-0.286803,0.957990,1.000000,0,0
1,0.198171,0.980168,1.714500,0.641673,0.766979,2.091667,0,1
2,0.530881,0.847446,0.836500,-0.897181,0.441662,1.341000,0,0
3,-0.636976,0.770884,3.121667,-0.439260,0.898360,5.797000,1,1
4,0.342020,0.939693,7.773333,0.334355,0.942447,8.680000,1,1


In [34]:
data = final_dataset.iloc[:, 0:6].to_numpy()
labels = final_dataset.iloc[:, 6:8].to_numpy()

In [13]:
class MyMLP(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(input_size, 8)
        self.fc2 = nn.Linear(8, 16) 
        self.fc3 = nn.Linear(16, output_size)

    def forward(self, x):
        out = self.relu(self.fc1(x))
        out = self.relu(self.fc2(out))
        out = self.fc3(out)
        return out

In [14]:
def train(n_epochs, optimizer, model, train_loader, loss_function):
    n_batch = len(train_loader)
    losses_train = []
    model.train()

    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for data, labels in train_loader:
            # Move data to device and convert to float
            data = data.to(dtype=torch.float, device=device)
            labels = labels.to(dtype=torch.float, device=device)  # Labels are binary (float)

            # Forward pass: compute predicted outputs
            output = model(data)

            # Compute loss (using BCEWithLogitsLoss or BCELoss depending on your model)
            loss = loss_function(output, labels)

            # Backpropagation and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()

        losses_train.append(loss_train / n_batch)

        if epoch == 1 or epoch % (n_epochs // 10) == 0:
            print('{}  |  Epoch {}  |  Training loss {:.3f}'.format(
                datetime.now().time(), epoch, loss_train / n_batch))

    return losses_train

In [15]:
def validate(model, val_loader, loss_function):
    model.eval()  # Set model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():  # No need to compute gradients during validation
        for data, labels in val_loader:
            # Move data to device
            data = data.to(dtype=torch.float, device=device)
            labels = labels.to(dtype=torch.float, device=device)  # Labels are binary (float)
            
            # Forward pass
            outputs = model(data)

            # Compute validation loss
            loss = loss_function(outputs, labels)
            val_loss += loss.item()

            # Compute accuracy (for each binary output)
            predictions = torch.sigmoid(outputs) >= 0.5  # Sigmoid and threshold at 0.5
            
            total += labels.size(0) * labels.size(1)  # Total number of binary labels
            correct += (predictions == labels).sum().item()  # Count correct predictions

    avg_val_loss = val_loss / len(val_loader)
    accuracy = correct / total  # Accuracy calculation across all binary labels
    
    return avg_val_loss, accuracy

In [16]:
n_models = 4

modelparams = {
  f"model{i}": {
    "lr": 0.025,
    "momentum": 0.9,
    "weight_decay": 0
  } for i in range(n_models)
}

modelparams["model1"].update({
    "lr": 0.025,
    "momentum": 0.50,
    "weight_decay": 0.0005,
})

modelparams["model2"].update({
    "lr": 0.05,
    "momentum": 0.90,
    "weight_decay": 0.0025,
})
modelparams["model3"].update({
    "lr": 0.025,
    "weight_decay": 0.001,
    "momentum" : 0.95
})

In [17]:
def make_dataloader(data, labels, batch_size = 64, shuffle = True):
    data_tensor = torch.from_numpy(data).float()
    labels_tensor = torch.from_numpy(labels).float()
    
    dataset = TensorDataset(data_tensor, labels_tensor)
    
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

In [18]:
train_data, test_val_data, train_labels, test_val_labels = train_test_split(
    data, labels, test_size=0.3, random_state=42
)
test_data, val_data, test_labels, val_labels = train_test_split(
    test_val_data, test_val_labels, test_size=0.5, random_state=42
)

batch_size = 64
train_loader = make_dataloader(train_data, train_labels, batch_size=batch_size, shuffle=True)
val_loader = make_dataloader(val_data, val_labels, batch_size=batch_size, shuffle=False)

In [19]:
first_third_start = np.quantile(test_data[:, 2], 0.33)
second_third_start = np.quantile(test_data[:, 2], 0.66)

first_third_end = np.quantile(test_data[:, 5], 0.33)
second_third_end = np.quantile(test_data[:, 5], 0.66)

print("3rd quantiles")
print(f"Start: \n1st {first_third_start}, 2nd {second_third_start}")
print(f"End: \n1st {first_third_end}, 2nd {second_third_end}")

mask_start = (test_data[:, 2] + test_data[:, 5]) < (first_third_end + first_third_start)
mask_end = (test_data[:, 2] + test_data[:, 5]) > (second_third_end + second_third_start)
mask_mid =  ~mask_start & ~mask_end

small_test_data, small_test_labels = test_data[mask_start], test_labels[mask_start]
medium_test_data, medium_test_labels = test_data[mask_mid], test_labels[mask_mid]
large_test_data, large_test_labels = test_data[mask_end], test_labels[mask_end]

small_test_loader = make_dataloader(small_test_data, small_test_labels, batch_size=batch_size, shuffle=False)
medium_test_loader = make_dataloader(medium_test_data, medium_test_labels, batch_size=batch_size, shuffle=False)
large_test_loader = make_dataloader(large_test_data, large_test_labels, batch_size=batch_size, shuffle=False)

3rd quantiles
Start: 
1st 2.483432, 2nd 3.68008
End: 
1st 2.5017866666666664, 2nd 3.8655999999999997


In [20]:
trainedmodels = {}
losses = {}
loss_function = nn.BCEWithLogitsLoss()

for name, params in modelparams.items():
  model = MyMLP(len(train_data[0]), len(train_labels[0]))
  model.to(device = device)
  optimizer = optim.SGD(model.parameters(), **params)

  print(f"{name}: ", "  |  ".join(map(lambda x: f"{str(x[0])}: {str(x[1])}",params.items())))
  losses[name]=train(100, optimizer, model, train_loader, loss_function)
  trainedmodels[name] = model

model0:  lr: 0.025  |  momentum: 0.9  |  weight_decay: 0
23:23:07.611364  |  Epoch 1  |  Training loss 0.680
23:23:08.450705  |  Epoch 10  |  Training loss 0.424
23:23:09.226098  |  Epoch 20  |  Training loss 0.422
23:23:09.992796  |  Epoch 30  |  Training loss 0.419
23:23:10.927033  |  Epoch 40  |  Training loss 0.412
23:23:11.757669  |  Epoch 50  |  Training loss 0.410
23:23:12.518307  |  Epoch 60  |  Training loss 0.413
23:23:13.277831  |  Epoch 70  |  Training loss 0.410
23:23:14.019733  |  Epoch 80  |  Training loss 0.405
23:23:14.574784  |  Epoch 90  |  Training loss 0.408
23:23:15.188199  |  Epoch 100  |  Training loss 0.407
model1:  lr: 0.025  |  momentum: 0.5  |  weight_decay: 0.0005
23:23:15.235456  |  Epoch 1  |  Training loss 0.690
23:23:15.731723  |  Epoch 10  |  Training loss 0.455
23:23:16.384466  |  Epoch 20  |  Training loss 0.430
23:23:17.001853  |  Epoch 30  |  Training loss 0.426
23:23:17.582167  |  Epoch 40  |  Training loss 0.418
23:23:18.120020  |  Epoch 50  |  T

In [21]:
best_acc = 0
best_model = ""
for name, model in trainedmodels.items():
    _, accuracy = validate(model, val_loader, loss_function)
    if accuracy > best_acc:
        best_model = name
        best_acc = accuracy

In [22]:
print(f"best model: {best_model}, {best_acc}")

best model: model2, 0.7939958592132506


In [23]:
for i, loader in enumerate([small_test_loader, medium_test_loader, large_test_loader]):
    _, accuracy = validate(trainedmodels[best_model], loader, loss_function)
    print(f"({i}/3 - {i+1}/3)  :  {accuracy}")

(0/3 - 1/3)  :  0.7547770700636943
(1/3 - 2/3)  :  0.8
(2/3 - 3/3)  :  0.8881987577639752


#### Results get more accurate as the wind is stronger! 

In [24]:
from sklearn.neural_network import MLPClassifier
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.base import BaseEstimator

models = {
    "Base line" : [DummyClassifier(strategy = "most_frequent"), {
    
    }],
    "Multi-layer Perception" : [MLPClassifier(random_state=42, max_iter = 1000), {
        "hidden_layer_sizes" : [10, 25, 50], 
    }],
    "K Nearest Neighbors" : [KNeighborsClassifier(), {
        "n_neighbors" : [5, 10, 20, 40], 
        "p": [1, 2, 3]
    }],
    "Random Forrest Classifier" : [RandomForestClassifier(random_state=42), {
        "max_depth": [2, 3], 
        "n_estimators" : [50, 100, 300]
    }],
    "Decision Tree Classifier" : [DecisionTreeClassifier(random_state=42), {
        "min_samples_split" : [2, 3, 4],
        "min_samples_leaf" : [1, 2, 3]
    }],
}

In [25]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

datalist = []

for i in models.keys():
    #does the gridsearch on the models
    model = GridSearchCV(models[i][0], models[i][1])
    #makes a prediction
    prediction = model.fit(train_data, train_labels).predict(test_data)
    #finds out how good that prediction is
    
    total = len(test_labels) * len(test_labels[0])  # Total number of binary labels
    correct = (prediction == test_labels).sum()
    accuracy = correct/total
    datalist.append([model.best_estimator_, accuracy])

In [26]:
for i, j in datalist:
    print(f"{i.__repr__(): <70}: {j: .3}")

DummyClassifier(strategy='most_frequent')                             :  0.55
MLPClassifier(hidden_layer_sizes=50, max_iter=1000, random_state=42)  :  0.811
KNeighborsClassifier(n_neighbors=40, p=3)                             :  0.822
RandomForestClassifier(max_depth=3, n_estimators=50, random_state=42) :  0.821
DecisionTreeClassifier(min_samples_leaf=2, random_state=42)           :  0.751


In [27]:
best_model = sorted(datalist, key=lambda x: x[1], reverse=True)[0][0]

In [30]:
for i, (data, labels) in enumerate([(small_test_data, small_test_labels), (medium_test_data, medium_test_labels), (large_test_data,large_test_labels)]):
    prediction = best_model.predict(data)
    
    total = len(labels) * len(labels[0])  # Total number of binary labels
    correct = (prediction == labels).sum()
    accuracy = correct/total
    
    print(f"({i}/3 - {i+1}/3)  :  {accuracy: .3}")

(0/3 - 1/3)  :   0.758
(1/3 - 2/3)  :   0.818
(2/3 - 3/3)  :   0.888
